In [1]:
#libraries

import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
from sklearn.preprocessing import MinMaxScaler
from keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.dates as mandates
from sklearn import linear_model
from keras.models import Sequential
from keras.layers import Dense

import keras.backend as K
from keras.callbacks import EarlyStopping
#from Keras.models import load_model
from keras.layers import LSTM
from keras.utils.vis_utils import plot_model
import tensorflow as tf
import keras

In [2]:
#Get the Dataset
df=pd.read_csv(r'C:\Users\varun\Downloads\Qmca9Y4Bp6s2FkVfRTNS6zZU4HvBd24sXgR2RsR1uBP5Re\dataset\train_dataset.csv')
df.head()

,date,symbol,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_2003,feature_2004,feature_2005,feature_2006,feature_2007,feature_2008,feature_2009,feature_DC1,target,target_DC1
0,2018-04-29,BTC,0.000000,0.000000,0.965517,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.689655,0.655172,0.655172,0.724138,0.827586,0.827586,0.931034,0.448276,0.758621,0.024718
1,2018-04-29,DLT,0.310345,0.310345,0.137931,0.275862,0.206897,0.517241,0.344828,0.689655,...,0.206897,0.206897,0.379310,0.172414,0.241379,0.379310,0.241379,0.793103,0.517241,-0.005240
2,2018-04-29,ENJ,0.241379,0.068966,0.310345,0.344828,0.103448,0.103448,0.379310,0.206897,...,0.379310,0.344828,0.310345,0.482759,0.655172,0.448276,0.724138,0.724138,0.413793,-0.036471
3,2018-04-29,OMG,0.758621,0.965517,0.896552,0.965517,0.965517,0.965517,0.068966,0.275862,...,0.896552,0.931034,0.896552,0.931034,0.896552,0.896552,0.827586,0.827586,0.275862,-0.066945
4,2018-04-29,UFR,0.620690,0.413793,0.275862,0.413793,0.517241,0.362069,0.758621,0.586207,...,0.482759,0.275862,0.172414,0.551724,0.103448,0.103448,0.137931,0.965517,0.724138,0.020286


In [3]:
symbols=df["symbol"].unique()
symbols


array(['BTC', 'DLT', 'ENJ', 'OMG', 'UFR', 'VGX', 'DRGN', 'BAT', 'AGIX',
       'KEY', 'KNC', 'TRAC', 'PRO', 'PLAY', 'PAY', 'DCN', 'DAY', 'CAN',
       'XRP', 'SRN', 'SALT', 'REQ', 'QSP', 'TRST', 'USDT', 'MGO', 'GET',
       'FUN', 'PRE', 'ETH', 'SPANK', 'ZRX', 'POLL', 'FUEL', 'ANT', 'AST',
       'SNGLS', 'VEE', 'VIB', 'PLR', 'BRD', 'WABI', 'MTL', 'NGC', 'IDH',
       'VIBE', 'PPT', 'DADI', 'DNT', 'GNO', 'POWR', 'DENT', 'GRID', 'OCN',
       'WPR', 'EVE', 'DNA', 'DGD', 'WTC', 'GVT', 'CDT', 'GLM', 'POLY',
       'RCN', 'EXRN', 'MTN', 'TKN', 'AXP', 'MANA', 'CRPT', 'HGT', 'REM',
       'ABT', 'RDN', 'LOC', 'LUN', 'MCO', 'PRG', 'ENG', 'CND', 'TNB',
       'PLBT', 'TEL', 'ELF', 'ATL', 'MKR', 'PPP', 'UTK', 'POE', 'NCASH',
       'BKX', 'ELEC', 'STORM', 'SNT', 'CHP', 'QASH', 'CAS', 'SHIP', 'SAN',
       'BNT', 'DMT', 'STAR', 'BANCA', 'ZAP', 'IND', 'BERRY', 'SAI', 'B2B',
       'BLZ', 'RFR', 'NCT', 'HT', 'TEN', 'ODE', 'CBT', 'BAX', 'DATA',
       'SENC', 'MWAT', 'DXT', 'SMT', 'LINK', 'CRC', 'M

In [4]:
count=0
for (uniquerow), group in df.groupby(df.symbol):
    count+=1
    globals()[uniquerow] = group

In [5]:
check = df[df.symbol.isin(symbols[0:10]) != False]


In [6]:
check

,date,symbol,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_2003,feature_2004,feature_2005,feature_2006,feature_2007,feature_2008,feature_2009,feature_DC1,target,target_DC1
0,2018-04-29,BTC,0.000000,0.000000,0.965517,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.689655,0.655172,0.655172,0.724138,0.827586,0.827586,0.931034,0.448276,0.758621,0.024718
1,2018-04-29,DLT,0.310345,0.310345,0.137931,0.275862,0.206897,0.517241,0.344828,0.689655,...,0.206897,0.206897,0.379310,0.172414,0.241379,0.379310,0.241379,0.793103,0.517241,-0.005240
2,2018-04-29,ENJ,0.241379,0.068966,0.310345,0.344828,0.103448,0.103448,0.379310,0.206897,...,0.379310,0.344828,0.310345,0.482759,0.655172,0.448276,0.724138,0.724138,0.413793,-0.036471
3,2018-04-29,OMG,0.758621,0.965517,0.896552,0.965517,0.965517,0.965517,0.068966,0.275862,...,0.896552,0.931034,0.896552,0.931034,0.896552,0.896552,0.827586,0.827586,0.275862,-0.066945
4,2018-04-29,UFR,0.620690,0.413793,0.275862,0.413793,0.517241,0.362069,0.758621,0.586207,...,0.482759,0.275862,0.172414,0.551724,0.103448,0.103448,0.137931,0.965517,0.724138,0.020286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86541,2022-08-14,DRGN,0.334052,0.441092,0.692529,0.262213,0.709052,0.660201,0.272989,0.354885,...,0.748563,0.777299,0.750000,0.775862,0.619253,0.570402,0.655172,0.926724,0.066092,-0.288638
86543,2022-08-14,BTC,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.998563,0.000000,...,0.155172,0.225575,0.250000,0.192529,0.343391,0.387931,0.308908,0.627155,0.612069,-0.121726
86558,2022-08-14,KEY,0.237787,0.424569,0.168103,0.278736,0.066092,0.298851,0.526580,0.401580,...,0.762931,0.770115,0.735632,0.798851,0.836207,0.791667,0.863506,0.538793,0.568966,-0.131796
86559,2022-08-14,AGIX,0.908764,0.915230,0.168103,0.714080,0.933908,0.922414,0.925287,0.943966,...,0.662356,0.727011,0.745690,0.685345,0.295977,0.326149,0.271552,0.869253,0.090517,-0.258920


In [7]:
len(pd.unique(check['symbol']))

10

In [8]:
BTC

,date,symbol,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_2003,feature_2004,feature_2005,feature_2006,feature_2007,feature_2008,feature_2009,feature_DC1,target,target_DC1
0,2018-04-29,BTC,0.0,0.0,0.965517,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.689655,0.655172,0.655172,0.724138,0.827586,0.827586,0.931034,0.448276,0.758621,0.024718
84,2018-05-06,BTC,1.0,0.0,0.981481,1.000000,0.981481,0.981481,0.000000,0.000000,...,0.388889,0.555556,0.537037,0.481481,0.777778,0.759259,0.722222,0.722222,0.685185,-0.101384
140,2018-05-13,BTC,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.426471,0.500000,0.397059,0.500000,0.750000,0.735294,0.705882,0.705882,0.838235,-0.024447
185,2018-05-20,BTC,0.0,0.0,0.988636,0.000000,0.011364,0.988636,0.988636,0.000000,...,0.250000,0.363636,0.397727,0.318182,0.522727,0.579545,0.500000,0.806818,0.863636,-0.144448
272,2018-05-27,BTC,0.0,0.0,0.000000,0.989362,0.000000,0.000000,0.989362,0.978723,...,0.095745,0.212766,0.287234,0.191489,0.595745,0.606383,0.553191,0.872340,0.361702,0.046671
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83961,2022-07-17,BTC,1.0,1.0,1.000000,0.997122,1.000000,0.000000,1.000000,1.000000,...,0.548201,0.588489,0.615827,0.572662,0.755396,0.775540,0.717986,0.404317,0.635971,0.083959
84474,2022-07-24,BTC,0.0,1.0,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,...,0.381089,0.538682,0.561605,0.517192,0.603152,0.630372,0.578797,0.636103,0.474212,0.031658
85376,2022-07-31,BTC,0.0,0.0,0.998586,0.998586,0.000000,0.000000,0.998586,0.000000,...,0.265912,0.404526,0.458274,0.352192,0.519095,0.570014,0.438472,0.472419,0.335219,-0.007029
86082,2022-08-07,BTC,1.0,0.0,0.000000,1.000000,1.000000,0.998584,0.000000,1.000000,...,0.205382,0.296034,0.327195,0.254958,0.454674,0.498584,0.389518,0.337110,0.629603,0.048287


In [9]:
ATN

,date,symbol,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_2003,feature_2004,feature_2005,feature_2006,feature_2007,feature_2008,feature_2009,feature_DC1,target,target_DC1
4976,2018-12-16,ATN,0.631696,0.502232,0.616071,0.680804,0.676339,0.651786,0.718750,0.468750,...,0.571429,0.401786,0.392857,0.424107,0.500000,0.491071,0.513393,0.799107,0.763393,0.334990
5159,2018-12-23,ATN,0.393939,0.534632,0.376623,0.517316,0.597403,0.547619,0.556277,0.616883,...,0.238095,0.337662,0.359307,0.333333,0.367965,0.367965,0.350649,0.761905,0.627706,-0.028946
5536,2018-12-30,ATN,0.740506,0.654008,0.759494,0.662447,0.784810,0.784810,0.740506,0.736287,...,0.337553,0.434599,0.468354,0.379747,0.459916,0.497890,0.426160,0.624473,0.109705,-0.095611
5649,2019-01-06,ATN,0.265690,0.495816,0.380753,0.525105,0.395397,0.514644,0.564854,0.606695,...,0.564854,0.514644,0.539749,0.485356,0.573222,0.606695,0.539749,0.112971,0.769874,-0.148778
5898,2019-01-13,ATN,0.754167,0.514583,0.733333,0.639583,0.762500,0.608333,0.708333,0.787500,...,0.362500,0.370833,0.354167,0.362500,0.454167,0.437500,0.454167,0.770833,0.383333,0.023228
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51123,2021-06-20,ATN,0.731141,0.701161,0.698259,0.774662,0.754352,0.863636,0.882012,0.841393,...,0.967118,0.920696,0.808511,0.965184,0.655706,0.411992,0.936170,0.945841,0.628627,-0.142997
52078,2021-06-27,ATN,0.275047,0.507561,0.480151,0.412098,0.517958,0.584121,0.739130,0.748582,...,0.956522,0.969754,0.971645,0.962193,0.911153,0.905482,0.918715,0.635161,0.164461,-0.028105
52136,2021-07-04,ATN,0.505703,0.324144,0.517110,0.474335,0.395437,0.489544,0.581749,0.713878,...,0.954373,0.956274,0.952471,0.950570,0.802281,0.752852,0.842205,0.165399,0.005703,-1.170049
53138,2021-07-11,ATN,0.625472,0.626415,0.416981,0.628302,0.611321,0.502830,0.575472,0.680189,...,0.973585,0.954717,0.937736,0.966038,0.318868,0.190566,0.616981,0.005660,0.215094,-0.213517


In [10]:
del df

In [11]:
import warnings
warnings.filterwarnings("ignore")

In [12]:
cor=[[] for i in range(980)]

In [13]:
for j in range (0,980):
    for i in range (0,2009):
        cor[j].append(globals()[symbols[j]]['feature_'+ str(i+1)].corr(globals()[symbols[j]]['target']))

In [14]:
index=[[] for i in range(980)]

In [15]:
for i in range (980):
    index[i] = np.abs(np.array(cor[i]))
    index[i] = np.argpartition(cor[i], -15)[-15:]

In [16]:
cor[0]

[0.057589542003998725,
 -0.005911653864813144,
 0.008987180323918815,
 0.003205220104856446,
 0.09642164276120502,
 -0.015515723908313168,
 -0.004038524837560707,
 0.07119726591394655,
 0.009125472482731293,
 -0.001326433331739107,
 0.016225912903367667,
 -0.013255302071795016,
 0.077683267032771,
 -0.04785798506868303,
 -0.011597151700131845,
 0.036768088288706736,
 0.08960054638424045,
 0.039404302576653935,
 0.041654848908281694,
 0.011972306388100169,
 0.0006257622347860883,
 0.06470159729763877,
 -0.015473019413535774,
 -0.05438038318676204,
 -0.028128421325542163,
 0.04436758053920576,
 0.016603841525204693,
 0.03208361691693379,
 0.07963125784059667,
 0.06695741816831485,
 -0.04264698411636608,
 -0.08244789308314414,
 -0.025188415669267574,
 -0.11750988314140418,
 0.022213609200879645,
 0.05778342120242963,
 0.008865125636847982,
 0.03523978823924835,
 -0.0023125342850659793,
 0.07457548823456916,
 -0.03842364498835132,
 -0.11387200575029761,
 -0.10828340378120815,
 -0.033886546

In [17]:
index[1]

array([1594, 1005,  445, 1582, 1550, 1007, 1008, 1720,  768, 1562, 1614,
        767, 1610, 1599, 1004], dtype=int64)

In [18]:

feature=[[] for i in range(980)]
for i in range(980):
    for j in index[i]:
        feature[i].append('feature_'+str(j+1))
        
        
        

In [19]:
len(feature[0])

15

In [20]:
feature[18]

['feature_618',
 'feature_1539',
 'feature_892',
 'feature_1629',
 'feature_1281',
 'feature_1011',
 'feature_442',
 'feature_1538',
 'feature_233',
 'feature_608',
 'feature_237',
 'feature_1756',
 'feature_239',
 'feature_1758',
 'feature_1765']

In [21]:
arr=[]

In [22]:
for i in symbols:
    arr.append('model_'+str(i))

In [23]:
arr[1]

'model_DLT'

In [24]:
for i in feature[0]:
    print(i)

feature_1399
feature_1398
feature_440
feature_58
feature_1408
feature_1389
feature_1400
feature_1742
feature_1017
feature_1410
feature_1751
feature_1753
feature_1406
feature_1780
feature_1778


In [25]:
del cor

In [26]:
del index

In [27]:
def model(i,X_train,y_train):
    arr[i] = Sequential()
    arr[i].add(LSTM(64, input_shape=(1, trainX.shape[1]), activation='relu', return_sequences=True))
    arr[i].add(LSTM(64, input_shape=(1, trainX.shape[1]), return_sequences=True))
    arr[i].add(LSTM(8, input_shape=(1, trainX.shape[1]), return_sequences=False))
    arr[i].add(Dense(1))
    arr[i].compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0014))
    history=arr[i].fit(X_train, y_train, epochs=100, batch_size=64, verbose=1, shuffle=False)   

In [28]:

for i in range(3):
    features = feature[i]
    target=['target']
    scaler = MinMaxScaler()
    feature_transform = scaler.fit_transform(globals()[symbols[i]][features])
    X_train= pd.DataFrame(columns=features, data=feature_transform, index=globals()[symbols[i]].index)    
    y_train= pd.DataFrame(columns=target, data=globals()[symbols[i]], index=globals()[symbols[i]].index)   
    trainX =np.array(X_train)
    X_train = trainX.reshape(X_train.shape[0], 1, X_train.shape[1])
    model(i,X_train,y_train)
    del X_train
    del y_train
    del trainX


Epoch 1/100
4/4 [==============================] - 4s 8ms/step - loss: 0.3674
Epoch 2/100
4/4 [==============================] - 0s 7ms/step - loss: 0.3458
Epoch 3/100
4/4 [==============================] - 0s 7ms/step - loss: 0.3209
Epoch 4/100
4/4 [==============================] - 0s 9ms/step - loss: 0.2894
Epoch 5/100
4/4 [==============================] - 0s 11ms/step - loss: 0.2477
Epoch 6/100
4/4 [==============================] - 0s 10ms/step - loss: 0.1924
Epoch 7/100
4/4 [==============================] - 0s 11ms/step - loss: 0.1240
Epoch 8/100
4/4 [==============================] - 0s 8ms/step - loss: 0.0569
Epoch 9/100
4/4 [==============================] - 0s 8ms/step - loss: 0.0262
Epoch 10/100
4/4 [==============================] - 0s 9ms/step - loss: 0.0403
Epoch 11/100
4/4 [==============================] - 0s 7ms/step - loss: 0.0386
Epoch 12/100
4/4 [==============================] - 0s 7ms/step - loss: 0.0265
Epoch 13/100
4/4 [==============================] - 0s 7ms

In [29]:
train=pd.read_csv(r'C:\Users\varun\Downloads\Qmca9Y4Bp6s2FkVfRTNS6zZU4HvBd24sXgR2RsR1uBP5Re\dataset\validation_dataset.csv')


In [30]:
train

,date,symbol,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_2003,feature_2004,feature_2005,feature_2006,feature_2007,feature_2008,feature_2009,feature_DC1,target,target_DC1
0,2022-08-21,BTC,0.001408,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.998592,...,0.184507,0.245070,0.274648,0.225352,0.345070,0.385915,0.312676,0.611268,NaN,NaN
1,2022-08-21,XRP,0.000000,0.001408,0.001408,0.001408,0.001408,0.001408,0.000000,0.000000,...,0.390141,0.478873,0.512676,0.438028,0.618310,0.640845,0.583099,0.733803,NaN,NaN
2,2022-08-21,SRN,0.465493,0.472535,0.533099,0.229577,0.497183,0.310563,0.517606,0.052817,...,0.969014,0.974648,0.969014,0.973239,0.981690,0.980282,0.980282,0.700000,NaN,NaN
3,2022-08-21,SALT,0.626761,0.714789,0.690845,0.702113,0.657042,0.702817,0.305634,0.652817,...,0.801408,0.873239,0.884507,0.846479,0.921127,0.928169,0.897183,0.863380,NaN,NaN
4,2022-08-21,REQ,0.852817,0.044366,0.085915,0.218310,0.284507,0.602817,0.014085,0.152817,...,0.633803,0.705634,0.732394,0.660563,0.802817,0.812676,0.769014,0.695775,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
706,2022-08-21,GAL,0.347887,0.050000,0.053521,0.940845,0.925352,0.933099,0.909859,0.934507,...,0.538028,0.329577,0.330986,0.328169,0.281690,0.297183,0.277465,0.304225,NaN,NaN
707,2022-08-21,SOFI,0.079577,0.745070,0.841549,0.702113,0.793662,0.731690,0.708451,0.748592,...,0.736620,0.740845,0.716901,0.752113,0.784507,0.735211,0.814085,0.821127,NaN,NaN
708,2022-08-21,QOM,0.238028,0.909859,0.928873,0.863380,0.679577,0.895775,0.854930,0.273944,...,0.659155,0.269014,0.307042,0.230986,0.145070,0.161972,0.125352,0.570423,NaN,NaN
709,2022-08-21,AVT,0.177465,0.314789,0.059155,0.257042,0.236620,0.107746,0.361972,0.092254,...,0.747887,0.187324,0.077465,0.807042,0.326761,0.140845,0.909859,0.738028,NaN,NaN


In [31]:
train["symbol"].nunique()

711

In [32]:
j=0

In [33]:
BTC

,date,symbol,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_2003,feature_2004,feature_2005,feature_2006,feature_2007,feature_2008,feature_2009,feature_DC1,target,target_DC1
0,2018-04-29,BTC,0.0,0.0,0.965517,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.689655,0.655172,0.655172,0.724138,0.827586,0.827586,0.931034,0.448276,0.758621,0.024718
84,2018-05-06,BTC,1.0,0.0,0.981481,1.000000,0.981481,0.981481,0.000000,0.000000,...,0.388889,0.555556,0.537037,0.481481,0.777778,0.759259,0.722222,0.722222,0.685185,-0.101384
140,2018-05-13,BTC,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.426471,0.500000,0.397059,0.500000,0.750000,0.735294,0.705882,0.705882,0.838235,-0.024447
185,2018-05-20,BTC,0.0,0.0,0.988636,0.000000,0.011364,0.988636,0.988636,0.000000,...,0.250000,0.363636,0.397727,0.318182,0.522727,0.579545,0.500000,0.806818,0.863636,-0.144448
272,2018-05-27,BTC,0.0,0.0,0.000000,0.989362,0.000000,0.000000,0.989362,0.978723,...,0.095745,0.212766,0.287234,0.191489,0.595745,0.606383,0.553191,0.872340,0.361702,0.046671
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83961,2022-07-17,BTC,1.0,1.0,1.000000,0.997122,1.000000,0.000000,1.000000,1.000000,...,0.548201,0.588489,0.615827,0.572662,0.755396,0.775540,0.717986,0.404317,0.635971,0.083959
84474,2022-07-24,BTC,0.0,1.0,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,...,0.381089,0.538682,0.561605,0.517192,0.603152,0.630372,0.578797,0.636103,0.474212,0.031658
85376,2022-07-31,BTC,0.0,0.0,0.998586,0.998586,0.000000,0.000000,0.998586,0.000000,...,0.265912,0.404526,0.458274,0.352192,0.519095,0.570014,0.438472,0.472419,0.335219,-0.007029
86082,2022-08-07,BTC,1.0,0.0,0.000000,1.000000,1.000000,0.998584,0.000000,1.000000,...,0.205382,0.296034,0.327195,0.254958,0.454674,0.498584,0.389518,0.337110,0.629603,0.048287


In [34]:
BTC

,date,symbol,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_2003,feature_2004,feature_2005,feature_2006,feature_2007,feature_2008,feature_2009,feature_DC1,target,target_DC1
0,2018-04-29,BTC,0.0,0.0,0.965517,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.689655,0.655172,0.655172,0.724138,0.827586,0.827586,0.931034,0.448276,0.758621,0.024718
84,2018-05-06,BTC,1.0,0.0,0.981481,1.000000,0.981481,0.981481,0.000000,0.000000,...,0.388889,0.555556,0.537037,0.481481,0.777778,0.759259,0.722222,0.722222,0.685185,-0.101384
140,2018-05-13,BTC,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.426471,0.500000,0.397059,0.500000,0.750000,0.735294,0.705882,0.705882,0.838235,-0.024447
185,2018-05-20,BTC,0.0,0.0,0.988636,0.000000,0.011364,0.988636,0.988636,0.000000,...,0.250000,0.363636,0.397727,0.318182,0.522727,0.579545,0.500000,0.806818,0.863636,-0.144448
272,2018-05-27,BTC,0.0,0.0,0.000000,0.989362,0.000000,0.000000,0.989362,0.978723,...,0.095745,0.212766,0.287234,0.191489,0.595745,0.606383,0.553191,0.872340,0.361702,0.046671
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83961,2022-07-17,BTC,1.0,1.0,1.000000,0.997122,1.000000,0.000000,1.000000,1.000000,...,0.548201,0.588489,0.615827,0.572662,0.755396,0.775540,0.717986,0.404317,0.635971,0.083959
84474,2022-07-24,BTC,0.0,1.0,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,...,0.381089,0.538682,0.561605,0.517192,0.603152,0.630372,0.578797,0.636103,0.474212,0.031658
85376,2022-07-31,BTC,0.0,0.0,0.998586,0.998586,0.000000,0.000000,0.998586,0.000000,...,0.265912,0.404526,0.458274,0.352192,0.519095,0.570014,0.438472,0.472419,0.335219,-0.007029
86082,2022-08-07,BTC,1.0,0.0,0.000000,1.000000,1.000000,0.998584,0.000000,1.000000,...,0.205382,0.296034,0.327195,0.254958,0.454674,0.498584,0.389518,0.337110,0.629603,0.048287


In [35]:
symbols1=train["symbol"].unique()
symbols1


array(['BTC', 'XRP', 'SRN', 'SALT', 'REQ', 'QSP', 'USDT', 'GET', 'FUN',
       'ETH', 'PAY', 'PLAY', 'PRO', 'TRAC', 'KNC', 'KEY', 'AGIX', 'BAT',
       'CHAT', 'HT', 'DRGN', 'VGX', 'OMG', 'DCN', 'ENJ', 'PLR', 'VEE',
       'ANT', 'AST', 'BRD', 'ZRX', 'VIB', 'WABI', 'OCN', 'CRPT', 'VIBE',
       'PPT', 'MTL', 'DNT', 'GNO', 'POWR', 'DENT', 'GRID', 'WPR', 'TKN',
       'STAR', 'MTN', 'RCN', 'POLY', 'GLM', 'REM', 'NCASH', 'ABT', 'AXP',
       'GVT', 'MANA', 'DGD', 'RDN', 'UTK', 'MKR', 'ELF', 'POE', 'TNB',
       'TEL', 'CND', 'ENG', 'CHP', 'SNT', 'TEN', 'STORM', 'ELEC', 'QASH',
       'CAS', 'BFT', 'SAN', 'BNT', 'DMT', 'ZAP', 'FDZ', 'NCT', 'RFR',
       'BLZ', 'MTH', 'GEM', 'LINK', 'SENC', 'DATA', 'BAX', 'RUFF', 'HOT',
       'CV', 'CEL', 'RLC', 'IHT', 'NEXO', 'GEN', 'CNN', 'DGX', 'LBA',
       'REN', 'FACE', 'QKC', 'ADB', 'MYB', 'CBC', 'LYM', 'OAX', 'XBP',
       'MET', 'COV', 'STORJ', 'LET', 'BOB', 'MFT', 'REP', 'AOA', 'HSC',
       'BMX', 'CXO', 'SEELE', 'COFI', 'DX', 'UBEX', 'HIT', 'OR

In [36]:
for i in range(711):
    symbols1[i]=(symbols1[i]+str(i))

In [37]:
symbols1

array(['BTC0', 'XRP1', 'SRN2', 'SALT3', 'REQ4', 'QSP5', 'USDT6', 'GET7',
       'FUN8', 'ETH9', 'PAY10', 'PLAY11', 'PRO12', 'TRAC13', 'KNC14',
       'KEY15', 'AGIX16', 'BAT17', 'CHAT18', 'HT19', 'DRGN20', 'VGX21',
       'OMG22', 'DCN23', 'ENJ24', 'PLR25', 'VEE26', 'ANT27', 'AST28',
       'BRD29', 'ZRX30', 'VIB31', 'WABI32', 'OCN33', 'CRPT34', 'VIBE35',
       'PPT36', 'MTL37', 'DNT38', 'GNO39', 'POWR40', 'DENT41', 'GRID42',
       'WPR43', 'TKN44', 'STAR45', 'MTN46', 'RCN47', 'POLY48', 'GLM49',
       'REM50', 'NCASH51', 'ABT52', 'AXP53', 'GVT54', 'MANA55', 'DGD56',
       'RDN57', 'UTK58', 'MKR59', 'ELF60', 'POE61', 'TNB62', 'TEL63',
       'CND64', 'ENG65', 'CHP66', 'SNT67', 'TEN68', 'STORM69', 'ELEC70',
       'QASH71', 'CAS72', 'BFT73', 'SAN74', 'BNT75', 'DMT76', 'ZAP77',
       'FDZ78', 'NCT79', 'RFR80', 'BLZ81', 'MTH82', 'GEM83', 'LINK84',
       'SENC85', 'DATA86', 'BAX87', 'RUFF88', 'HOT89', 'CV90', 'CEL91',
       'RLC92', 'IHT93', 'NEXO94', 'GEN95', 'CNN96', 'DGX97', 'LBA9

In [38]:
j=0

In [39]:
for (uniquerow), group in train.groupby(train.symbol):
    globals()[symbols1[j]] = group
    j=j+1    

In [55]:
for i in range(3):
    features = feature[i]
    target=['target']
    scaler = MinMaxScaler()
    feature_transform = scaler.fit_transform(globals()[symbols[i]][features])
    globals()[symbols1[i]]= pd.DataFrame(columns=feature[i], data=feature_transform, index=globals()[symbols[i]].index)
    # validation_dataset.head()

    # if this doesnt work maybe we can 

In [56]:
globals()[symbols1[0]]

,feature_1399,feature_1398,feature_440,feature_58,feature_1408,feature_1389,feature_1400,feature_1742,feature_1017,feature_1410,feature_1751,feature_1753,feature_1406,feature_1780,feature_1778
0,0.530538,0.469902,0.949871,0.000000,0.707151,0.450343,0.452944,0.569748,0.793103,0.371419,0.601314,0.489877,0.490033,0.000000,0.094767
84,0.646889,0.629406,1.000000,0.981481,0.442888,0.657203,0.704481,0.553863,0.981481,0.095367,0.802716,0.744260,0.352779,0.592421,0.647445
140,0.708914,0.740207,0.925640,0.000000,0.548238,0.720217,0.682334,0.803460,1.000000,0.598926,0.850815,0.805013,0.180256,0.749474,0.795806
185,0.742746,0.833158,0.893698,1.000000,0.662498,0.783804,0.782560,0.724691,1.000000,0.486010,0.908909,0.903539,0.521801,0.835139,0.871182
272,0.817392,0.823197,0.817777,1.000000,0.414531,0.830425,0.720502,0.791170,0.000000,0.581308,0.961780,0.940460,0.317121,0.967837,0.976342
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83961,0.467257,0.486895,0.382397,0.000000,0.753796,0.459910,0.506280,0.356087,0.933813,0.641780,0.429250,0.385913,0.214725,0.515590,0.540099
84474,0.442650,0.458198,0.370925,0.998567,0.650781,0.431291,0.426831,0.635078,0.000000,0.384699,0.694940,0.668375,0.600688,0.511929,0.531818
85376,0.405877,0.399066,0.402266,0.000000,0.249424,0.415984,0.422577,0.498545,0.049505,0.602817,0.469294,0.489035,0.232180,0.560240,0.586110
86082,0.372080,0.375572,0.389154,0.997167,0.768302,0.374371,0.413845,0.225057,0.000000,0.761237,0.373188,0.335907,0.780666,0.279360,0.324085


In [42]:
# for i in range(3):
#     globals()[symbols1[i]]=  globals()[symbols1[i]].drop(['date'], axis=1)
#     globals()[symbols1[i]]=  globals()[symbols1[i]].drop(['symbol'], axis=1)
#     # del globals()[symbols[i]+str(i)]
#     # del globals()[symbols[i]]





KeyError: "['date'] not found in axis"

In [53]:
globals()[symbols1[0]]

array([[[0.53053814, 0.4699019 , 0.9498708 , ..., 0.4900328 ,
         0.        , 0.09476719]],

       [[0.6468889 , 0.62940586, 1.        , ..., 0.35277924,
         0.5924213 , 0.6474455 ]],

       [[0.70891416, 0.7402074 , 0.92563957, ..., 0.18025623,
         0.7494741 , 0.7958055 ]],

       ...,

       [[0.40587658, 0.39906603, 0.4022663 , ..., 0.23218046,
         0.56024   , 0.58611   ]],

       [[0.37208024, 0.37557185, 0.38915437, ..., 0.78066564,
         0.27936012, 0.32408485]],

       [[0.33783215, 0.32190853, 0.44721678, ..., 0.33020467,
         0.4383117 , 0.4166953 ]]], dtype=float32)

In [54]:
for i in range(3):
    validation_X =np.array(globals()[symbols1[i]])
    validation_X = validation_X.reshape(validation_X.shape[0], 1, validation_X.shape[1])
    globals()[symbols1[i]]=  np.asarray(validation_X).astype('float32')


ValueError: cannot reshape array of size 3375 into shape (225,1,1)

In [48]:
 globals()[symbols1[0]]

array([[[0.53053814, 0.4699019 , 0.9498708 , ..., 0.4900328 ,
         0.        , 0.09476719]],

       [[0.6468889 , 0.62940586, 1.        , ..., 0.35277924,
         0.5924213 , 0.6474455 ]],

       [[0.70891416, 0.7402074 , 0.92563957, ..., 0.18025623,
         0.7494741 , 0.7958055 ]],

       ...,

       [[0.40587658, 0.39906603, 0.4022663 , ..., 0.23218046,
         0.56024   , 0.58611   ]],

       [[0.37208024, 0.37557185, 0.38915437, ..., 0.78066564,
         0.27936012, 0.32408485]],

       [[0.33783215, 0.32190853, 0.44721678, ..., 0.33020467,
         0.4383117 , 0.4166953 ]]], dtype=float32)

In [49]:
y_prediction=[]


In [50]:
arr[i]

In [51]:
for i in range(3):
    y_prediction.append(arr[i].predict(globals()[symbols1[i]]))

    
    

8/8 [==============================] - 1s 4ms/step


ValueError: in user code:

    File "C:\Users\varun\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\engine\training.py", line 1845, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\varun\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\engine\training.py", line 1834, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\varun\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\engine\training.py", line 1823, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\varun\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\engine\training.py", line 1791, in pr

In [ ]:
y_prediction

[array([[0.7205328 ],
        [0.6561731 ],
        [0.7081581 ],
        [0.7473461 ],
        [0.6519256 ],
        [0.7189167 ],
        [0.66286546],
        [0.78618675],
        [0.63139385],
        [0.6297495 ],
        [0.82034165],
        [0.85960597],
        [0.7278438 ],
        [0.76134914],
        [0.76659334],
        [0.72945166],
        [0.65922266],
        [0.6373985 ],
        [0.7103476 ],
        [0.51774096],
        [0.5419445 ],
        [0.5110749 ],
        [0.58013445],
        [0.63542116],
        [0.5379318 ],
        [0.5541741 ],
        [0.5470294 ],
        [0.563707  ],
        [0.5340899 ],
        [0.70883286],
        [0.63325125],
        [0.731885  ],
        [0.6370193 ],
        [0.5497747 ],
        [0.4969275 ],
        [0.51515967],
        [0.56016976],
        [0.52431434],
        [0.3875042 ],
        [0.5581819 ],
        [0.48915544],
        [0.59171575],
        [0.5566003 ],
        [0.60398024],
        [0.5106037 ],
        [0

In [ ]:
y_pred=pd.DataFrame(y_prediction)

In [ ]:
# validation_dataset1=pd.read_csv(r'C:\Users\varun\Downloads\QmbVhN84RZYuZ33JMrua8ngtnKcgVsbgM89rU6MTnhDuQa\dataset\train_dataset.csv')
# )
prediction = pd.DataFrame()
prediction['symbol'] = list(symbols[0:3])
prediction['prediction'] = y_pred

In [ ]:
prediction

,symbol,prediction
0,BTC,"[[0.6539397], [0.6455718], [0.6723911], [0.752..."
1,DLT,"[[0.47277415], [0.6121403], [0.41993135], [0.3..."
2,ENJ,"[[0.60855013], [0.5032341], [0.71712744], [0.2..."
3,OMG,NaN


In [ ]:
# prediction.to_csv(r'"C:\Users\varun\OneDrive\Documents\model.csv', index = False, header = None)